In [228]:
import pandas as pd
from pathlib import Path

# Data Loading

In [229]:
# dir = Path("~/Documents/Kaggle_Data")
dir = Path("~/Data")
dir = dir / "CMI-Wrist"
# dir = Path("~/Data")
# dir = dir / "CMI-Wrist"

In [ ]:
# train_data_csv = dir / "train.csv"
# train_demo_csv = dir / "train_demographics.csv"
# df = pd.read_csv(train_data_csv)
# df.head()

In [ ]:
# df_d = pd.read_csv(train_demo_csv)
# df_d.head()

In [ ]:
# df.to_pickle(dir / "train.pkl")
# df_d.to_pickle(dir / "train_demo.pkl")

In [ ]:
# merging
# df_m = pd.merge(df,df_d,how="left",on="subject")
# df_m.head()

In [ ]:
# df_m.shape, df.shape

In [ ]:
# df_m.to_pickle(dir / "train_merged.pkl")

# Data Basic Splitting
Will use holdout method first for looking at basics of the dataset. Will split this into train and val sets in the hopes of preventing data leakage. Will eventually use stratified k-folds for training.

The data also contains sequences. These will need to be preserved in order to train. So, splitting by subject and/or sequence_id will be best. You will also need to make sure that gestures are stratified between train and validation. That way train and val datasets have examples of the all gestures.

Splitting by subject alone will may cause some gestures to be left out. However, one benefit of splitting by subject is that it may promote training for generality. If a subject is in both the training and validation datasets, the model will have been exposed to all nuances of an individual subject. This would make the model overfit the data. If split by subject, then the model will need to learn general characteristics of a movement in order to correctly label the validation set. If k-folds are done, then the model will have access to all subjects and gestures, but it will be forced to validate itself against unseen subjects, which should promote generality.

In [230]:
df_m = pd.read_pickle(dir / "train.pkl")

In [231]:
df_y = df_m['gesture']
df_X = df_m.drop(columns=['gesture'])

In [ ]:
# Getting df of just subject


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df_X, df_y, stratify=df_y, random_state=123)

In [ ]:
X_train.to_pickle(dir / "X_train_explore.pkl")
y_train.to_pickle(dir / "y_train_explore.pkl")
X_val.to_pickle(dir / "X_val_explore.pkl")
y_val.to_pickle(dir / "y_val_explore.pkl")